In [10]:
import brawlstats
from pathlib import Path
import pandas as pd

token = open(Path('G:/내 드라이브/GitHub/Brawl Stars Stats/brawlstats_token_raon.txt')).read().strip()

In [11]:
# 클래스를 객체화합니다.
client = brawlstats.Client(token)

In [12]:
# 랭커의 데이터를 수집합니다. brawlstats.Client 클래스에서는 1~200위 랭커의 정보를 제공하고 있습니다.
rankers = client.get_rankings(ranking='players')
len(rankers)

200

In [13]:
# 랭커들의 태그를 추출해서 리스트로 변환합니다.
rankers_tag_list = [rankers[i]['tag'].split("#")[1] for i in range(len(rankers))]
rankers_tag_list

['YGU20CRG',
 '8820PPUC',
 'VGPLRCVQ',
 'VLQPVPY',
 'QCC0UVC',
 'G2JQC0U',
 '8URCLRY8',
 '8LQ9JR82',
 '82UCLY9C8',
 '2QU0QU2VQ',
 'JGUYRYQ',
 '2PGGR8Y9P',
 'CJV2PJ0R',
 '8YL8889QP',
 'CPL9PLC8',
 'QRU0L0U',
 'CG0YQP2C',
 '9P2CUYJ8P',
 'UYUUQQ8Q',
 '802VG829',
 '2YCR9RLCJ',
 '20VYV2CQJ',
 '2R2CVPJYJ',
 'V2Q080UU',
 'LLQ8GY8',
 'UVVULJUQ',
 '2GJVUCL',
 '9C9YJ22YL',
 'JLJ2LQ0Y',
 '8GUPLYY',
 '2RG0LV02',
 '99YL0PV9V',
 '92C8J829Q',
 'QQPJ2QJ9',
 'P99QGPLU2',
 'QLQCLGYV2',
 'Y989PY8Q',
 '80YGP0YQ',
 '90220GRPU',
 'CL2RUY8Y',
 '8JVV0P808',
 '2JJ9G0UP2',
 'GJ9PLGPL',
 '29J2GYQ2',
 '89JUJUVYL',
 '2LL9008LV',
 '2L892GP',
 '8U2GYYCR9',
 '2UYJV98J2',
 'JGCCGY80',
 '28PP0C82J',
 'VCJGV08V',
 '29JJCQULC',
 'QUYCVC2',
 '9Q9C9JU0',
 'PR9U2JL',
 '2P0QU92P2',
 'LJCL8P0',
 'PP9R0PQ8J',
 '8QPQUJUPU',
 'VYR0RR0P',
 'CGQCYJGY',
 'QGG2UUV0',
 '9P98UCUV',
 'VY20VCCY',
 'Y989VP',
 'V92PGJPL',
 '280RJ92JC',
 '2VPLLJJ0V',
 '8QVGJQGGC',
 '808YCJQ0',
 '8V92UYCJ',
 '2P2URUV8G',
 'C2V8LPQQ',
 'QURVLPG',
 'YQLY9CYVY

In [14]:
# Client 클래스의 get_profile 함수를 이용하면 해당 유저의 브롤러와 가젯유무, 스타파워유무 등을 알 수 있습니다.
# 1위 랭커의 profile을 살펴봅니다.
tag = rankers_tag_list[0]
user_info = client.get_profile(tag)
user_info.brawlers

BoxList([{'id': 16000000, 'name': 'SHELLY', 'power': 11, 'rank': 35, 'trophies': 1302, 'highest_trophies': 1302, 'gears': [{'id': 62000002, 'name': 'DAMAGE', 'level': 3}, {'id': 62000000, 'name': 'SPEED', 'level': 3}, {'id': 62000004, 'name': 'SHIELD', 'level': 3}], 'star_powers': [{'id': 23000076, 'name': 'SHELL SHOCK'}, {'id': 23000135, 'name': 'BAND-AID'}], 'gadgets': [{'id': 23000255, 'name': 'FAST FORWARD'}, {'id': 23000288, 'name': 'CLAY PIGEONS'}]}, {'id': 16000001, 'name': 'COLT', 'power': 11, 'rank': 35, 'trophies': 1344, 'highest_trophies': 1344, 'gears': [{'id': 62000002, 'name': 'DAMAGE', 'level': 3}, {'id': 62000004, 'name': 'SHIELD', 'level': 3}, {'id': 62000005, 'name': 'RELOAD SPEED', 'level': 3}], 'star_powers': [{'id': 23000077, 'name': 'SLICK BOOTS'}, {'id': 23000138, 'name': 'MAGNUM SPECIAL'}], 'gadgets': [{'id': 23000273, 'name': 'SPEEDLOADER'}, {'id': 23000319, 'name': 'SILVER BULLET'}]}, {'id': 16000002, 'name': 'BULL', 'power': 11, 'rank': 35, 'trophies': 1302, 

In [15]:
# get_battle_logs 결괏값의 구조를 확인합니다.
battles = client.get_battle_logs(tag)
battle = battles[0]

print(battle)
print()
print(battle.keys())

{'battle_time': '20230304T192821.000Z', 'event': {'id': 15000368, 'mode': 'knockout', 'map': "Belle's Rock"}, 'battle': {'mode': 'knockout', 'type': 'ranked', 'result': 'victory', 'duration': 126, 'trophy_change': 8, 'star_player': {'tag': '#8GVQPJV', 'name': 'NAVI | MMA', 'brawler': {'id': 16000022, 'name': 'TICK', 'power': 11, 'trophies': 829}}, 'teams': [[{'tag': '#YGU20CRG', 'name': 'vital shark', 'brawler': {'id': 16000065, 'name': 'MANDY', 'power': 11, 'trophies': 175}}, {'tag': '#8GVQPJV', 'name': 'NAVI | MMA', 'brawler': {'id': 16000022, 'name': 'TICK', 'power': 11, 'trophies': 829}}, {'tag': '#8P99C8VC', 'name': 'CrySoul', 'brawler': {'id': 16000001, 'name': 'COLT', 'power': 11, 'trophies': 897}}], [{'tag': '#2GPJJRJRC', 'name': 'CSS|Losn', 'brawler': {'id': 16000046, 'name': 'BELLE', 'power': 11, 'trophies': 919}}, {'tag': '#29Y2VYY9L', 'name': '彡•Շσɾɬίx͒❄️乂', 'brawler': {'id': 16000048, 'name': 'GROM', 'power': 11, 'trophies': 966}}, {'tag': '#8U0C2GVCQ', 'name': 'T0sha', 'b

In [16]:
# 배틀정보를 딕셔너리로 만들고, value만 가져와서 리스트로 변환합니다.
# 다음 블로그를 참고해서 함수를 수정했습니다. (https://assaeunji.github.io/data%20analysis/2020-05-20-brawlstats/)


def parse_single_user(tag):
    parsed_logs = []
    battles = client.get_battle_logs(tag)
    for battle in battles:

        # 배틀모드가 3대3인 경우만 추출합니다. 그외 클럽리그, 파워리그, 보스전, 빅게임, 로보럼블까지 총 5개의 모드는 배틀로그 형태가 달라서 일단 제외합니다.
        if battle['event']['mode'] in ['gemGrab', 'brawlBall', 'knockout', 'heist', 'bounty', 'hotZone', 'basketBrawl']:

            battle_info = {
                "battle_time": battle['battle_time'],
                "mode": battle['event']['mode'],
                "map": battle['event']['map'],
                "type": battle['battle']['type'],
                "duration": battle['battle']['duration']
            }

            # 배틀결과가 무승부인 경우와 같이 스타플레이어가 없는 경우 오류를 피하기 위해 if 조건문을 사용합니다.
            if battle['battle']['star_player'] != None:
                star_player_tag = battle['battle']['star_player']['tag'][1:]

                teams = battle['battle']['teams']

                for team in teams:
                    players_tag = [member['tag'][1:] for member in team]
                    side = 1 if tag in players_tag else 0

                    for player in team:
                        player_info = client.get_profile(player['tag'][1:])
                        player_brawlers = sorted(player_info.brawlers, key= lambda x: x['trophies'], reverse=True)

                        for brawler in player_brawlers:
                            if brawler['id']==player['brawler']['id']:
                                gadget = 1 if ['gadgets'] else 0
                                star_power = 1 if brawler['star_powers'] else 0
                        
                        team_info = {
                            'player_tag': player['tag'],
                            'player_name': player['name'],
                            'player_trophies': player_info.trophies,
                            'player_top_brawler': player_brawlers[0]['name'],
                            'star_player': 1 if player['tag'][1:] == star_player_tag else 0,
                            'team': side,
                            'result': battle['battle']['result'],
                            'brawler_id': player['brawler']['id'],
                            'brawler_name': player['brawler']['name'],
                            'brawler_power': player['brawler']['power'],
                            'brawler_star_power': star_power,
                            'brawler_gadget': gadget,
                            'brawler_trophies': player['brawler']['trophies'],
                        }

                        parsed_logs.append({**battle_info, **team_info})
            
            
            else:
                teams = battle['battle']['teams']

                for team in teams:
                    players_tag = [member['tag'][1:] for member in team]
                    side = 1 if tag in players_tag else 0

                    for player in team:
                        player_info = client.get_profile(player['tag'][1:])
                        player_brawlers = sorted(player_info.brawlers, key= lambda x: x['trophies'], reverse=True)

                        for brawler in player_brawlers:
                            if brawler['id']==player['brawler']['id']:
                                gadget = 1 if ['gadgets'] else 0
                                star_power = 1 if brawler['star_powers'] else 0
                        
                        team_info = {
                            'player_tag': player['tag'],
                            'player_name': player['name'],
                            'player_trophies': player_info.trophies,
                            'player_top_brawler': player_brawlers[0]['name'],
                            'star_player':None,
                            'team': side,
                            'result': battle['battle']['result'],
                            'brawler_id': player['brawler']['id'],
                            'brawler_name': player['brawler']['name'],
                            'brawler_power': player['brawler']['power'],
                            'brawler_star_power': star_power,
                            'brawler_gadget': gadget,
                            'brawler_trophies': player['brawler']['trophies'],
                        }

                        parsed_logs.append({**battle_info, **team_info})

    return parsed_logs

오류 : `TypeError: 'NoneType' object is not subscriptable`
- 오류 내용을 구글링
- 원인 : bounty 모드에서는 스타플레이어가 없다. 그래서 battle['battle']['star_player']가 None 값이었다.
- 해결 : 모드가 bounty 이면 star_player 정보를 넣지 않는다.

오류 : `TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"`
- 오류 내용을 구글링
- 원인 : 데이터프레임이 여러개인데 리스트로 넣지 않아서.. 간단하군!
- 해결 : 데이터프레임을 리스트로 묶어준다. `pd.concat([df1, df2], axis=0, sort='battle_time')` 그럼 해결!

클럽리그, 파워리그, 보스전, 빅게임, 로보럼블 없음.

<h1>데이터 합치기</h1>

In [17]:
'''# 시험삼아 1위 랭커와 2위 랭커의 배틀정보를 합칩니다.
df1 = pd.DataFrame(parse_single_user(rankers_tag_list[0]))
df2 = pd.DataFrame(parse_single_user(rankers_tag_list[1]))

pd.concat([df1, df2], axis=0, sort='battle_time')'''

"# 시험삼아 1위 랭커와 2위 랭커의 배틀정보를 합칩니다.\ndf1 = pd.DataFrame(parse_single_user(rankers_tag_list[0]))\ndf2 = pd.DataFrame(parse_single_user(rankers_tag_list[1]))\n\npd.concat([df1, df2], axis=0, sort='battle_time')"

In [18]:
# 빈 데이터프레임을 만듭니다.
columns = pd.DataFrame(parse_single_user(rankers_tag_list[0])).columns
df = pd.DataFrame(columns=columns)
df

,battle_time,mode,map,type,duration,player_tag,player_name,player_trophies,player_top_brawler,star_player,team,result,brawler_id,brawler_name,brawler_power,brawler_star_power,brawler_gadget,brawler_trophies


In [19]:
# 1~200위 랭커의 정보를 가져와서 데이터프레임을 채워줍니다.
for ranker in rankers_tag_list:
    try:
        # df = pd.concat([df, pd.DataFrame(parse_single_user(ranker))], join='full', axis=0, sort='battle_time')
        df = pd.concat([df, pd.DataFrame(parse_single_user(ranker))], axis=0, ignore_index=True)
    except:
        pass

In [ ]:
df[:20]

,battle_time,mode,map,type,duration,player_tag,player_name,player_trophies,player_top_brawler,star_player,team,result,brawler_id,brawler_name,brawler_power,brawler_star_power,brawler_gadget,brawler_trophies
0,20230304T192821.000Z,knockout,Belle's Rock,ranked,126,#YGU20CRG,vital shark,82315,ASH,0,1,victory,16000065,MANDY,11,1,1,175
1,20230304T192821.000Z,knockout,Belle's Rock,ranked,126,#8GVQPJV,NAVI | MMA,54356,OTIS,1,1,victory,16000022,TICK,11,1,1,829
2,20230304T192821.000Z,knockout,Belle's Rock,ranked,126,#8P99C8VC,CrySoul,43562,COLT,0,1,victory,16000001,COLT,11,1,1,897
3,20230304T192821.000Z,knockout,Belle's Rock,ranked,126,#2GPJJRJRC,CSS|Losn,56788,TARA,0,0,victory,16000046,BELLE,11,1,1,919
4,20230304T192821.000Z,knockout,Belle's Rock,ranked,126,#29Y2VYY9L,彡•Շσɾɬίx͒❄️乂,51257,GENE,0,0,victory,16000048,GROM,11,1,1,966
5,20230304T192821.000Z,knockout,Belle's Rock,ranked,126,#8U0C2GVCQ,T0sha,48634,BONNIE,0,0,victory,16000058,BONNIE,11,1,1,897
6,20230304T192536.000Z,knockout,Belle's Rock,ranked,73,#YGU20CRG,vital shark,82315,ASH,0,1,defeat,16000065,MANDY,11,1,1,177
7,20230304T192536.000Z,knockout,Belle's Rock,ranked,73,#8P99C8VC,CrySoul,43562,COLT,0,1,defeat,16000001,COLT,11,1,1,907
8,20230304T192536.000Z,knockout,Belle's Rock,ranked,73,#8GVQPJV,NAVI | MMA,54356,OTIS,0,1,defeat,16000022,TICK,11,1,1,838
9,20230304T192536.000Z,knockout,Belle's Rock,ranked,73,#C00P92QG,goody,52582,SAM,0,0,defeat,16000060,SAM,11,1,1,949


오류 : csv 파일로 저장 시 player_trophies 열의 값이 사라지고 하나씩 열이 당겨지는 현상이 발생한다.

- 원인 : player_name 문자열 가장 끝의 값이 이모티콘인 경우 발생하는 듯하다.
- 해결시도 :
1. player_name에서 이모티콘을 삭제한다. (플레이어의 태그로 식별 가능하기에 정보 자체에 문제가 발생하지는 않는다.)
2. 100% 문제해결을 위해 숫자, 영문, 한글이 아닌 경우 해당 문자를 삭제한다.

In [ ]:
df_origin = df.copy()

In [ ]:
# 정규표현식을 사용하여 특수기호를 제거합니다.
df["player_name"] = df["player_name"].str.replace(pat=r'[^\w]', repl=r'', regex=True)

# 정규표현식을 사용하여 숫자, 영문, 한글이 아닌 경우 제거합니다.
df["player_name"] = df["player_name"].str.replace(pat=r'[^1-9a-zA-Z가-힣]', repl=r'', regex=True)

In [ ]:
df[:20]

NameError: name 'df' is not defined

In [ ]:
df[df['star_player']==1]

,battle_time,mode,map,type,duration,player_tag,player_name,player_trophies,player_top_brawler,star_player,team,result,brawler_id,brawler_name,brawler_power,brawler_star_power,brawler_gadget,brawler_trophies
1,20230304T192821.000Z,knockout,Belle's Rock,ranked,126,#8GVQPJV,NAVIMMA,54356,OTIS,1,1,victory,16000022,TICK,11,1,1,829
10,20230304T192536.000Z,knockout,Belle's Rock,ranked,73,#PVRPPLLUL,StPFearLesS,46023,GUS,1,0,defeat,16000061,GUS,11,1,1,925
17,20230304T192351.000Z,knockout,Belle's Rock,ranked,53,#8GVQPJV,NAVIMMA,54356,OTIS,1,1,victory,16000022,TICK,11,1,1,831
22,20230304T192205.000Z,knockout,Belle's Rock,ranked,63,#8P99C8VC,CrySoul,43562,COLT,1,1,victory,16000001,COLT,11,1,1,894
28,20230304T192038.000Z,knockout,Belle's Rock,ranked,138,#8P99C8VC,CrySoul,43562,COLT,1,1,victory,16000001,COLT,11,1,1,887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17322,20230305T170409.000Z,brawlBall,Super Beach,ranked,150,#8LPL8RYLQ,Libertasatv,50206,GROM,1,1,victory,16000038,SURGE,11,1,1,954
17327,20230305T170108.000Z,brawlBall,Super Beach,ranked,60,#R9RRRPY,Patinkton,51802,BUSTER,1,1,victory,16000011,MORTIS,11,1,1,922
17336,20230305T165945.000Z,brawlBall,Super Beach,ranked,60,#R9RRRPY,Patinkton,51802,BUSTER,1,1,victory,16000011,MORTIS,11,1,1,916
17338,20230305T165810.000Z,brawlBall,Super Beach,ranked,80,#9RJPPGYUV,Zacca,60792,MAX,1,0,defeat,16000006,BARLEY,11,1,1,847


In [ ]:
df.to_csv('C:/Users/yenny/Desktop/23030611_rankers.csv', encoding='utf-8')